In [1]:
from glob import glob
import numpy as np
import cv2, os

In [2]:
pred_path = './pred.png'
gt_path = './gt.png'

In [3]:
def target_rgb2label(target_img, label_colors):
    width, height, _ = target_img.shape
    target_label = np.zeros([width, height], dtype=np.uint8)
    for it in range(1, len(label_colors)):
        rr, cc = np.where(np.all(target_img == label_colors[it], axis=-1))
        target_label[rr, cc] = it
    return target_label

In [11]:
pred = cv2.imread(pred_path)
gt = cv2.imread(gt_path)
color_map = np.array([[0, 0, 255], [255, 232, 131], [255, 0, 0],
                        [0, 255, 0], [131, 89, 161], [175, 89, 62]])


if not pred.shape == gt.shape:
    pred = cv2.resize(pred, gt.shape[1::-1], interpolation=cv2.INTER_NEAREST)

gt_ = target_rgb2label(gt[:,:,[2,1,0]], color_map)
pred_ = target_rgb2label(pred[:,:,[2,1,0]], color_map)

In [19]:
nc = len(color_map) - 1
confuse_matrix = np.zeros([nc, nc])

for it in range(nc):  # gt
    for jt in range(nc):  # pred
        confuse_matrix[it, jt] = np.sum(np.logical_and(gt_ == (it+1), pred_ == (jt+1)))


In [25]:
nc = len(color_map)
confuse_matrix = np.zeros([nc, nc])

for it in range(nc):  # gt
    for jt in range(nc):  # pred
        confuse_matrix[it, jt] = np.sum(np.logical_and(gt_ == it, pred_ == jt))



In [34]:
np.set_printoptions(suppress=True)
confuse_matrix

array([[617827.,   3786.,      0.,    476.,     23.,      5.],
       [  3791.,  23322.,      0.,     53.,     54.,    127.],
       [     0.,      0.,      0.,      0.,      0.,      0.],
       [   575.,     45.,      0.,   2489.,      0.,      5.],
       [     8.,     71.,      0.,      0.,   1669.,      0.],
       [   242.,    115.,      0.,      2.,      0.,   4729.]])

In [36]:
confusion_matrix = confuse_matrix
n = confusion_matrix.shape[0]
precisions, recalls = [], []
for it in range(1, n):
    recall = confusion_matrix[it, it] / np.sum(confusion_matrix[it, :] + 10**-5)
    precision = confusion_matrix[it, it] / np.sum(confusion_matrix[:, it] + 10**-5)
    recalls.append(recall)
    precisions.append(precision)

In [49]:
pp=[precisions, recalls]


In [54]:
print(confusion_matrix, end='\n\n')
print("precision")
print(pp[0])
print("recall")
print(pp[1])

[[617827.   3786.      0.    476.     23.      5.]
 [  3791.  23322.      0.     53.     54.    127.]
 [     0.      0.      0.      0.      0.      0.]
 [   575.     45.      0.   2489.      0.      5.]
 [     8.     71.      0.      0.   1669.      0.]
 [   242.    115.      0.      2.      0.   4729.]]

precision
[0.8530670452034083, 0.0, 0.824172169056182, 0.9558991653184707, 0.9718454462986588]
recall
[0.8528174918210755, 0.0, 0.7992934977657, 0.9548054592172038, 0.9294418129389724]


In [ ]:
cv2.imshow("test", gt)
cv2.waitKey(0)
cv2.destroyAllWindows()